# Jupipa - Modelo de otimização de rotas de carros-pipa utilizando Julia



### Resolvendo o modelo de programação linear utilizando o JuMP


Para resolver um problema de programação linear tradicional para o problema de otimização de distribuição:

$$
\begin{align}
& \text{minimizar} && c^T x \\
& \text{sujeito a} && A x \leq b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$
onde $c^T \in \mathbb{R}^n$ é a transformação linear que desejamos minimizar, ou custo, $A$ é uma matrix $n \times n$ que define as restrições do problema. 

A variável de decisão para este problema é
$$
\begin{align}
\forall (p,o,d) \in R \quad \left(x_{(p,o,d)}= \text{quantia do produto } p \text{ que é transportado da origem } o \text{ para o destino d} \right)
\end{align}
$$


In [2]:
using JuMP, DelimitedFiles, GLPK

In [30]:
oferta = readdlm("../dados/exemplo_oferta.txt");
demanda = readdlm("../dados/exemplo_demanda.txt");
Distancias = readdlm("../dados/exemplo_distancias.txt");
noferta = size(oferta,1)
ndemanda = size(demanda,1);

In [31]:
Distancias

5×3 Array{Float64,2}:
 1.0  10.0  2.0
 1.0  10.0  2.0
 1.0  10.0  2.0
 1.0  10.0  2.0
 1.0  10.0  2.0

In [32]:
#MODEL CONSTRUCTION
LpModel = Model(with_optimizer(GLPK.Optimizer)) 
#VARIABLES
@variable(LpModel, x[1:ndemanda*noferta] >= 0) # Models x >=0
#OBJECTIVE
@objective(LpModel, Min,sum(Distancias[i,j]*x[j+(i-1)*noferta] for i in 1:ndemanda for j in 1:noferta));

In [33]:
#CONSTRAINTS
#demanda e oferta
for i in 1:ndemanda
    @constraint(LpModel, sum(x[j+(i-1)*noferta] for j in 1:noferta) == demanda[i]) # the ith row 
end 
for j in 1:noferta
    @constraint(LpModel, sum(x[j+(i-1)*noferta] for i in 1:ndemanda) <= oferta[j]) # the ith row 
end

In [8]:
LpModel

A JuMP Model
Minimization problem with:
Variables: 15
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [34]:
#SOLVE IT
# finally optimize the model
@time begin
status = JuMP.optimize!(LpModel) # solves the model
end
# TEST SOLVER STATUSES
@show JuMP.has_values(LpModel)
@show JuMP.termination_status(LpModel)
@show JuMP.primal_status(LpModel) == MOI.FEASIBLE_POINT;

 26.149933 seconds (26.68 M allocations: 1.310 GiB, 6.47% gc time)
JuMP.has_values(LpModel) = true
JuMP.termination_status(LpModel) = OPTIMAL::TerminationStatusCode = 1
JuMP.primal_status(LpModel) == MOI.FEASIBLE_POINT = true


In [35]:
# DISPLAY THE RESULTS
#-------------------------------- 
println("Objective value: ", JuMP.objective_value(LpModel)) # JuMP.objectivevalue(model_name) gives the optimum objective value
#println("Optimal solution is x = \n", JuMP.value.(x)) # JuMP.resultvalue(decision_variable) will give the optimum value 

[println("destino: ",i,", origem: ",j,", carros: ",JuMP.value.(x)[j+(i-1)*noferta]) for i in 1:ndemanda for j in 1:noferta if JuMP.value.(x)[j+(i-1)*noferta] >0];

Objective value: 11.0
destino: 1, origem: 1, carros: 1.0
destino: 2, origem: 1, carros: 1.0
destino: 3, origem: 1, carros: 1.0
destino: 4, origem: 1, carros: 2.0
destino: 5, origem: 3, carros: 3.0
